In [1]:
import time         # 用於延遲顯示棋盤
import random       # 用於隨機選擇棋步
import pytest       # 用於測試
import numpy as np  # 用於棋盤表示和數值計算

# epsilon-greedy策略的探索率，越大越隨機
EPS = 0.1

# 學習率，控制狀態價值更新幅度
ALPHA = 0.1

# 最大學習回合數
MAX_EPISODE = 100000

# 是否在學習過程中顯示棋盤
SHOW_LEARN = False

# pytest fixture，用於測試時產生Game實例
@pytest.fixture
def g():
    return Game()

# 定義非法棋步異常，用於玩家輸入不合法時拋出
class IllegalPosition(Exception):
    pass

class Game(object):
    """井字棋遊戲類別"""

    def __init__(self):
        self.reset_state()   # 初始化棋盤
        self.st_values = {}  # 狀態價值函數字典，key = 棋盤狀態tuple，value = 勝率

    def reset_state(self):
        """重置棋盤為全空"""
        self.state = np.zeros(9)  # 0表示空格

    def query_state_value(self, _state = None):
        """
        查詢遊戲狀態的價值，如果尚未計算，則用calc_state_value計算。
        返回值:
            0: X 贏
            1: O 贏
            0.5: 平手或尚未決勝負
        """
        state = self.state if _state is None else _state  # 如果沒傳入state，使用當前棋盤
        tstate = tuple(state)                             # 將numpy array轉為tuple，方便作為字典key
        if tstate not in self.st_values:
            # 若狀態尚未計算過，計算並存入字典
            self.st_values[tstate] = calc_state_value(state)

        # 回傳狀態價值
        return self.st_values[tstate]

    def get_legal_index(self):
        """
        取得合法下棋位置索引列表
        空位置即合法
        """
        return np.nonzero(np.equal(0, self.state))[0].tolist()  # 返回棋盤為0的位置

    def get_user_input(self, test_input = None):
        """
        取得玩家輸入的位置，檢查合法性
        """
        idx = None

        # 測試模式可直接傳入值
        if test_input is not None:
            inp = test_input
        else:
            inp = input(self.user_input_prompt())  # 提示玩家輸入位置
        try:
            idx = int(inp) - 1                     # 將玩家輸入的1-9轉為0-8索引
        except ValueError:
            raise IllegalPosition()                # 非數字輸入拋出異常

        # 檢查索引合法性
        if idx < 0 or idx > 8:
            raise IllegalPosition()
        if idx not in self.get_legal_index():
            raise IllegalPosition()
        return idx  # 返回合法索引

    def user_input_prompt(self):
        """提示玩家輸入棋盤位置"""
        return "Enter position[1-9]: "

    def draw(self):
        """返回棋盤文字顯示"""
        return draw(self.state)

def egreedy_index(state, legal_indices, query_state_value, side, eps = EPS):
    """
    epsilon-greedy: 策略選擇下棋位置
    state: 當前棋盤
    legal_indices: 可下棋位置
    query_state_value: 查詢狀態價值函數
    side: 玩家(1 = X, 2 = O)
    eps: 探索機率
    """
    # 隨機選擇(探索)
    if random.random() < eps:
        return random.choices(legal_indices)[0]
    else:
        indices = []  # 用於存放最大價值的候選位置
        max_val = -1  # 最大價值初始化

        # 模擬雙方可能下法，選最大值
        for s in [side, 3 - side]:              # 先自己下，後對手下
            for li in legal_indices:            # 遍歷每個合法位置
                state[li] = s                   # 嘗試下棋
                val = query_state_value(state)  # 查詢狀態價值
                if s == 1:                      # X玩家，價值取1-val
                    val = 1 - val

                # 更新最大值候選
                if val > max_val:
                    indices = [li]
                    max_val = val
                elif val == max_val and val < 1.0:
                    indices.append(li)
                state[li] = 0  # 還原棋盤

        # 若有多個最大值，隨機選一個
        return random.choices(indices)[0]

def draw(state):
    """文字化顯示棋盤"""
    rv = '\n'

    for y in range(3):
        for x in range(3):
            idx = y * 3 + x  # 計算線性索引
            t = state[idx]
            if t == 1:
                rv += 'X'
            elif t == 2:
                rv += 'O'
            else:
                if x < 2:
                    rv += ' '

            if x < 2:
                rv += '|'
        rv += '\n'
        if y < 2:
            rv += '-----\n'
    return rv

def judge(g):
    """判斷是否有勝者或平手"""
    wside = get_win_side(g.state)  # 取得勝者
    finish = False

    if wside > 0:  # 有勝者
        print(g.draw())
        print_winner(wside)
        finish = True
    elif len(g.get_legal_index()) == 0:  # 平手
        print("Draw!")
        finish = True

    if finish:
        again = input("Play again?(y/n):")  # 詢問是否再玩
        if again.lower() != 'y':
            return True
        else:
            g.reset_state()
            return False

def play_turn(g, side):
    """
    執行一個回合
    side: 1 = 系統下棋, 2 = 玩家下棋
    """
    if side == 1:
        # 系統使用epsilon-greedy選擇位置
        idx = egreedy_index(g.state, g.get_legal_index(), g.query_state_value, side, 0)
    
    # 玩家下棋
    else:
        while True:
            try:
                idx = g.get_user_input()    # 取得玩家輸入
            except IllegalPosition:
                print("Illegal position!")  # 提示非法
            else:
                break

    g.state[idx] = side    # 落子
    print(g.draw())        # 顯示棋盤
    stop = judge(g)        # 判斷勝負或平手
    return 3 - side, stop  # 回傳下一玩家與是否結束

def play(_g = None):
    """玩家與系統對戰"""
    g = Game() if _g is None else _g
    side = 1  # 系統先手

    while True:
        side, stop = play_turn(g, side)
        if stop is not None:
            if stop:
                break
            else:
                if side == 2:
                    print(g.draw())
                continue

def print_winner(wside):
    """顯示勝者"""
    print("Winner is '{}'".format('O' if wside == 2 else 'X'))

def learn():
    """系統自我學習"""
    g = Game()
    side = 1
    wside = 0
    
    import os
    if os.path.exists("result.txt"):
        load_file(g)  # 檢查是否有result.txt，若有則載入
    else:
        # 否則開始自我對弈學習
        for e in range(MAX_EPISODE):
            lidx = g.get_legal_index()     # 取得合法位置
            if len(lidx) == 0:
                wside = -1                 # 平手
            else:
                eps = np.exp(-e * 0.0005)  # 隨著學習逐漸減少探索率
                wside, side = _learn_body(g, lidx, side, eps)

            if SHOW_LEARN:
                time.sleep(1)
                if wside > 0:
                    print_winner(wside)
                elif wside == -1:
                    print('Draw')

            # 如果遊戲結束，重置棋盤
            if wside > 0 or wside == -1:
                g.reset_state()
                if SHOW_LEARN:
                    time.sleep(1)
        save(g.st_values)  # 存檔

    g.reset_state()
    return g

def _learn_body(g, lidx, side, eps):
    """學習主體：選擇動作、更新價值函數"""
    state = tuple(g.state)         # 保存當前棋盤狀態

    # 選擇動作
    idx = egreedy_index(g.state, lidx, g.query_state_value, side, eps)
    value = g.query_state_value()  # 原狀態價值
    g.state[idx] = side            # 落子

    if SHOW_LEARN:
        print(g.draw())

    nvalue = g.query_state_value()                     # 新狀態價值
    g.st_values[state] = update_values(value, nvalue)  # 更新狀態價值
    wside = get_win_side(g.state)                      # 判斷是否勝利
    side = 3 - side                                    # 換人
    return wside, side                                 # 回傳勝者與下一玩家

def update_values(this_value, next_value):
    """使用TD方法更新狀態價值"""
    diff = next_value - this_value
    return this_value + ALPHA * diff

def calc_state_value(state):
    """計算棋盤狀態價值"""
    ws = get_win_side(state)

    if ws == 2:
        return 1
    elif ws == 1:
        return 0
    else:
        return 0.5

def save(st_values):
    """存檔狀態價值"""
    with open("result.txt", "w") as f:
        for state, value in st_values.items():
            f.write("{}: {}\n".format(state, value))

def load_file(g):
    """載入存檔"""
    g.st_values = {}
    with open("result.txt", "r") as f:
        list1 = f.readlines()
        for row in list1:
            state, value = row.split(': ',1)
            g.st_values[state] = float(value)

def get_win_side(_state):
    """判斷是否連成一線
    return 0: 無贏家
           1: X 贏
           2: O 贏
    """
    state = _state.copy().reshape((3, 3))  # 轉成3x3棋盤

    for s in [1, 2]:
        # 檢查行與列
        for t in range(2):
            for r in range(3):
                if np.array_equal(np.unique(state[r]), [s]):
                    return s
            state = state.transpose()

        # 檢查對角線
        if _state[0] == s and _state[4] == s and _state[8] == s:
            return s
        if _state[2] == s and _state[4] == s and _state[6] == s:
            return s

    return 0  # 無勝者

# 當此程式被直接執行，而不是被當作模組匯入時，下面的程式碼才會執行
if __name__ == "__main__":

    # 呼叫learn()函數，讓系統進行自我學習，生成或載入已學習好的狀態價值函數
    # learn()會回傳一個Game實例，其st_values已包含學習結果
    game = learn()

    # 呼叫play()函數，開始與使用者互動下棋
    # 使用者與系統交替下棋，直到勝負或平手
    play(game)


 | |X
-----
 | |
-----
 | |


 | |X
-----
 |O|
-----
 | |


 | |X
-----
 |O|
-----
 | |X

Illegal position!

O| |X
-----
 |O|
-----
 | |X


O| |X
-----
 |O|X
-----
 | |X


O| |X
-----
 |O|X
-----
 | |X

Winner is 'X'

 | |
-----
 | |
-----
 | |


 | |
-----
 | |O
-----
 | |


 | |
-----
 |X|O
-----
 | |


 | |
-----
 |X|O
-----
 |O|


 |X|
-----
 |X|O
-----
 |O|


 |X|
-----
O|X|O
-----
 |O|


 |X|X
-----
O|X|O
-----
 |O|


O|X|X
-----
O|X|O
-----
 |O|


O|X|X
-----
O|X|O
-----
X|O|


O|X|X
-----
O|X|O
-----
X|O|

Winner is 'X'

 | |
-----
 | |
-----
 | |


 | |
-----
 |O|
-----
 | |


 |X|
-----
 |O|
-----
 | |


 |X|
-----
 |O|
-----
 | |O


X|X|
-----
 |O|
-----
 | |O

Illegal position!

X|X|O
-----
 |O|
-----
 | |O


X|X|O
-----
 |O|X
-----
 | |O


X|X|O
-----
O|O|X
-----
 | |O


X|X|O
-----
O|O|X
-----
X| |O

Illegal position!

X|X|O
-----
O|O|X
-----
X|O|O

Draw!
